<h1> 📘 Biofilter — Reports</h1>


Biofilter provides a Report system designed for reproducible, structured queries over curated biological and operational data. Reports encapsulate:

* Input validation
* Query logic
* Post-processing
* Standardized outputs

They are ideal for:

* Annotations and Filtering
* Quality control
* ETL monitoring
* Reusable scientific workflows

All reports are managed through the ReportManager, accessible via the main Biofilter object.

Summary of ReportManager Methods

| Method               | Purpose                                        |
| -------------------- | ---------------------------------------------- |
| `list()`             | List all available reports                     |
| `run()`              | Execute a report and return a DataFrame        |
| `run_example()`      | Run a report using its example input           |
| `explain()`          | Show or return report documentation            |
| `get_report_class()` | Return the report class                        |
| `get_report()`       | Instantiate a report object                    |
| `refresh()`          | Clear caches and re-discover reports (dev use) |




When to Use Reports vs Queries

Reports
✔ Reusable
✔ Structured
✔ Documented
✔ Versionable

Queries
✔ Exploratory
✔ Ad-hoc
✔ Flexible

👉 Use Reports for standardized workflows and Queries for exploration.

--------

### Start Biofilter3R

In [6]:
from biofilter import Biofilter
bf = Biofilter()

[INFO] ════════════════════════════════════
[INFO] 🚀 Initializing Biofilter3R
[INFO]    • Version: 3.2.0
[INFO]    • Debug mode: False
[INFO]    • Config: /home/bioadmin/biofilter/.biofilter.toml
[INFO] ════════════════════════════════════
[INFO] 🔌 Database connection established
[INFO]    • Engine: postgresql+psycopg2
[INFO]    • Host:   localhost
[INFO]    • DB:     biofilter
[INFO]    • Time:   1.7 ms
[INFO] ════════════════════════════════════


------

### 1. Listing Available Reports

Use this as a discovery mechanism in notebooks and interactive sessions.

In [7]:
bf.report.list()


📄 Available Reports

1. entity_filter
   Validates input list of entity names and returns all matching entities, including conflict and status flags.
   module: report_entity_filter

2. etl_packages
   Shows detailed ETL package execution records, including extract / transform / load statuses, timestamps, row counts, hashes, and stats.
   module: report_etl_packages

3. etl_status
   Shows the latest successful (good) ETL packages per DataSource for extract/transform/load, highlighting stale steps when hashes do not align with the most recent extract.
   module: report_etl_status

4. gene_to_snp
   Given a list of genes, returns gene metadata and SNPs overlapping the gene genomic region (GRCh37/38) using VariantSNP.
   module: report_gene_to_snp

5. qry_template
   Describe what this report does here
   module: report_template



-------

### 2. Running a Report

The most common method is run(). This returns a pandas DataFrame with a consolidated view of the ETL pipeline state.

In [8]:
df = bf.report.run(
    "etl_status",
    source_system="ncbi",
    data_sources=["dbsnp_chr1", "dbsnp_chr2"],
)

### 2. Running a Report with Example Input

Some reports define an example_input() for quick testing and tutorials.

This is useful for:

Demos

Documentation

Verifying report behavior after schema changes

To run a report using its built-in example:

In [9]:
df = bf.report.run_example("gene_to_snp")

AttributeError: 'ReportManager' object has no attribute 'run_report'

### 2. Getting a Report Explanation

Reports may provide a detailed explanation describing:

Purpose

Inputs

Outputs

Example usage

To view it:

In [ ]:
# To view it:

bf.report.explain("etl_status")


# Or capture it as a string:

text = bf.report.explain("etl_status", print_output=False)

-----------

In [ ]:
Accessing the Report Class Directly

If you want to inspect or extend a report programmatically:

cls = bf.report.get_report_class("etl_status")
cls


This returns the report class, not an instance.

Useful for:

Introspection

Documentation generation

Testing

----------- 

In [ ]:
Instantiating a Report Object

You can also instantiate a report manually:

report = bf.report.get_report(
    "etl_status",
    source_system="NCBI",
    only_active=True,
)

df = report.run()


This is helpful when:

Running the same report multiple times

Debugging inside notebooks

Extending reports in custom workflows